In [44]:
import numpy as np
import pandas as pd
from scipy.integrate import cumulative_trapezoid
import matplotlib.pyplot as plt
from scipy.integrate import cumulative_trapezoid
import glob

In [68]:
# DO NOT RUN AGAIN
# We want to create 4 * sets of dummy data and store as csv for each of them 

# Create some x values, these will act as the x values for all of the following curves
x = np.linspace(0, 50, 10000)

# We want to create 4 sets of dummy data so use iteration 
for i in range(4):

    # Substitute the x values into this sin curve 
    y = np.sin(x)

    # We will want to add some random noise to each data point, this noise will be guassian: 
    y += np.random.normal(0,0.2, size=x.shape)

    # Now create a dataframe of this data: 
    df = pd.DataFrame({'time': x,
                       'voltage': y,
                       'current': y*0.1})

    # Now save this data to this location:
    df.to_csv(f'data_set{i}.csv', index=False)
    

# Initialize an empyt list to store selected DataFrames
data_frames = []

# Read in the dummy data sets
for i in range(4):
    data = pd.read_csv(f'data_set{i}.csv')

    # Add the data to one dataframe: 
    if i == 0:
        selected_data = data[['time','voltage', 'current']]
    else: 
        selected_data = data[['voltage', 'current']]

    data_frames.append(selected_data)

# Now combine all of the selected data into one DataFrame
combined = pd.concat(data_frames, axis = 1)

# Now relabel the columns to indicate which voltage is considered: 
combined.columns = ['Time (s)', 'Voltage 1 (V)', 'Voltage 2 (V)', 'Voltage 3 (V)', 'Voltage 4 (V)',
                    'Current 1 (A)', 'Current 1 (A)']

# For each point in time take an average of these data, make this a new data frame
selected_columns = combined[['Voltage 1 (V)', 'Voltage 2 (V)', 'Voltage 3 (V)', 'Voltage 4 (V)']]
combined['Average Voltage (V)'] = selected_columns.mean(axis=1)

# Add an an error on the average voltage, the error on this is given by
# the standard deviation of the values divided by the sqrt of how many there 
# are. 
combined['Average Voltage Error (V)'] = (selected_columns.std(axis=1)/np.sqrt(combined.shape[1] - 2))

# Select the time column from this 'combined' data frame, and the average column 
# and add it to a new dataframe, this will be clearer to manipulate
df = combined[['Time (s)', 'Average Voltage (V)', 'Average Voltage Error (V)']]

# Add in a column for the error on the time 
time_error = pd.Series([0.01] * len(df))
df.insert(1, 'Time Error (s)', time_error)

In [69]:
def read_in_files(file_path: str=None,
                  time_error: float=None):
    
    # Path to your CSV files (adjust this path to your actual file location)
    csv_files = glob.glob(file_path)

    # Initialize lists to accumulate values for averaging
    channel_b_sum = None
    channel_c_sum = None
    time_column = None

    # Iterate over each CSV file
    for i, file in enumerate(csv_files):
        # Read each file
        df = pd.read_csv(file)
        
        # Extract "Time (ms)", "Channel B (mV)", and "Channel C (mV)" columns
        time_column = df['Time (ms)'] / 1000  # Convert Time from ms to seconds (only needs to be done once)
        channel_b = df['Channel B (mV)'] / 1000  # Convert from mV to V
        channel_c = df['Channel C (mV)'] / 1000  # Convert from mV to V

        # Accumulate the values for averaging
        if channel_b_sum is None:
            channel_b_sum = channel_b
            channel_c_sum = channel_c
        else:
            channel_b_sum += channel_b
            channel_c_sum += channel_c

    # Calculate the averages by dividing by the number of files
    channel_b_avg = channel_b_sum / len(csv_files)
    channel_c_avg = channel_c_sum / len(csv_files)

        # Second pass: Calculate the squared deviations for standard deviation
    for file in csv_files:
        # Read each file
        df = pd.read_csv(file)
        
        # Convert from mV to V
        channel_b = df['Channel B (mV)'] / 1000
        channel_c = df['Channel C (mV)'] / 1000
        
        # Accumulate the squared deviations from the mean
        channel_b_sq_dev_sum += (channel_b - channel_b_avg) ** 2
        channel_c_sq_dev_sum += (channel_c - channel_c_avg) ** 2

    # Calculate the standard deviation by taking the square root of the average squared deviation
    channel_b_std = np.sqrt(channel_b_sq_dev_sum / len(csv_files))/np.sqrt(len(csv_files))
    channel_c_std = np.sqrt(channel_c_sq_dev_sum / len(csv_files))/np.sqrt(len(csv_files))

    # Combine the results into a single DataFrame with the time column
    result_df = pd.DataFrame({
        'Time (s)': time_column,
        'Channel B Average (V)': channel_b_avg,
        'Channel B Average Error (V)': channel_b_std,
        'Channel C Average (V)': channel_c_avg,
        'Channel C Average Error (V)': channel_c_std
    })

    # Need to add an error on the time in (s)
    time_error = pd.Series([time_error] * len(result_df))
    df.insert(1, 'Time Error (s)', time_error)
 
    return result_df


In [ ]:


# Create a function which will compute the error on the product of 2 variables 
def error_multi_product(A: np.array=None,
                        alpha_A: np.array=None,
                        B: np.array=None,
                        alpha_B: np.array=None) -> np.array:
    """
    This function calculates the error on the product of 2 variables A and B
    """

    return (A*B) * np.sqrt((alpha_A/A)**(2) + (alpha_B/B)**(2))

# We can create an integration function: 
def B_field_calculation(df: pd.DataFrame=None,
                          time_column: str=None,
                          voltage_column: str=None,
                          voltage_current_column: str=None,
                          B_0: float=None,
                          alpha_int_volt: float=None,
                          N_2: int=None,
                          A: float=None,
                          alpha_A: float=None) -> pd.DataFrame:
    """
    This function takes in a series of data points (x,y') and returns a pandas DataFrame
    of the points (x, y), where these points lie on the integral of the curve (x, y').

    Arguments: 
    df: The data frame which contains the initial (x,y') points to be integrated
    time_column: the name of the column which contains the time data
    voltage_column: the name of the column which contains the voltage data from the secondary coil
    voltage_current_column: the name of the column which contains the voltage data to be converted to current 
    B_0: the initial value of the mangnetic field.
    alpha_B_0: the error on the initial value of B_0
    N_2: the number of turns on the secondary coil
    A: Area of the coil in m^(2)
    alpha_A: Error on the area (m^(2))

    Returns: 
    The same pandas dataframe with an integral column appended. This column is the B-field.
    """  

    # Get pandas series of the x and y values: 
    x_values = df[time_column]
    y_values = df[voltage_column]

    # Define a numpy array to hold the integral values 
    integral_value = np.zeros(len(x_values))
    # Assign the first value of the B-field at t=0
    integral_value[0] = B_0

     # Create a copy of the dataframe to insert the B-field column to
    df_to_ret = df.copy()

    # Now iterate through the array len(x_values - 1) times:
    df_to_ret['Integrated Voltage (Vs)'] = cumulative_trapezoid(df['Average Voltage (V)'], df['Time (s)'], initial=B_0)

    # Now we want to calculate the errors on the B-field. This must be done in parts.
    # We will calculate the errors on each Integrated Voltage (Vs) which is B_1 and beyond. B_0's error is pre-defined by the user. 

    # Compute the sum of each (V_{i-1} + V_{i})/2:
    voltage_sums_2 = np.array(df_to_ret['Average Voltage (V)'][1:] + np.array(df_to_ret['Average Voltage (V)'][:-1]))/2

    # Take out the Average Voltage Error from the dataframe and store it as a numpy array
    average_voltage_error = np.array(df_to_ret['Average Voltage Error (V)'])
    # Create a numpy array to hold the errors on the sums of the voltages, this should be of length x_values - 1. 
    voltage_sum_error = np.sqrt(average_voltage_error[1:]**(2) + average_voltage_error[:-1]**(2))
    
    # The error on the voltage sum divided by 2 is then:
    voltage_sum_error_2 = voltage_sum_error/2

    # Replace all NaN values with 0.
    voltage_sum_error_2 = np.nan_to_num(voltage_sum_error_2, nan=0)

    # Compute the difference in the time values and make it an array
    time_diff = df_to_ret['Time (s)'][1] - df_to_ret['Time (s)'][0]
    time_diff = np.full(len(x_values) - 1, time_diff)
    
    # the error on the time difference will be the same for each error in B:
    alpha_time_diff = np.sqrt(2*(df_to_ret['Time Error (s)'][0])**(2))
    time_diff_error = np.full(len(x_values) - 1, alpha_time_diff)

    # Now we need the error on the product of the sum of the voltages and the time difference, made modulus:
    product_voltage_time_diff_error = abs(error_multi_product(A=voltage_sums_2, alpha_A=voltage_sum_error_2, B=time_diff, alpha_B=time_diff_error))

    # Calculate the errors on the B filed at points B_{0} and beyond
    integrated_voltage_err = np.zeros(len(x_values))
    # Assign the first error to that inputted to the function:
    integrated_voltage_err[0] = alpha_int_volt
    # Now calculate the remaining: 
    for i in range(1, len(x_values)):
        integrated_voltage_err[i] = np.sqrt((integrated_voltage_err[i-1])**(2) + (product_voltage_time_diff_error[i-1])**(2))

    df_to_ret['Integrated Voltage Error (Vs)'] = integrated_voltage_err

    # Add a column for the B filed
    df_to_ret['B Field (T)'] = df_to_ret['Integrated Voltage (Vs)']/(N_2*A)

    # Compute the error on the B field
    B_field_error = error_multi_product(A=np.array(df_to_ret['Integrated Voltage (Vs)']), 
                                        alpha_A=np.array(df_to_ret['Integrated Voltage Error (Vs)']),
                                        B = 1/A,
                                        alpha_B = alpha_A)

    # Add an error on the B fieled column
    df_to_ret['B Field Error (T)'] = B_field_error

    return df_to_ret


# Practice: 
B_field_calculation(df=df, 
                      x_column='Time (s)',
                      y_column = 'Average Voltage (V)',
                      B_0 = 0.001,
                      alpha_int_volt=0.1,
                      N_2=5,
                      A=0.001,
                      alpha_A=0.0001)
